Step 1: Connecting to Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from google.colab import files

Mounted at /content/drive


Step 2: Download the Alzheimer's Dataset directly from the Kaggle website.

In [ ]:
import os
! pip install -q kaggle

In [ ]:
cp /content/drive/MyDrive/kaggle.json /content


In [ ]:
! chmod 600 /content/kaggle.json
os.environ['KAGGLE_CONFIG_DIR'] = "/content/"
! kaggle datasets download tourist55/alzheimers-dataset-4-class-of-images
! unzip /content/alzheimers-dataset-4-class-of-images.zip

Step 3: Import the Requirements

In [ ]:
import numpy as np
import pandas as pd
import os
import copy
import warnings
warnings.filterwarnings('ignore')
import cv2
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img, img_to_array
import matplotlib
import matplotlib.pylab as plt
import numpy as np
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from keras.applications.vgg16 import VGG16,preprocess_input
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Reshape, Permute, multiply, Dense, Flatten, GlobalAveragePooling2D
import tensorflow as tf
import itertools


Step 4: Prepare the Training and the Validation Data for Training the network.
        Rescaling and Resizing the images are also performed in this step.

In [ ]:
data_dir = '/content/Alzheimer_s Dataset/train'
BATCH_SIZE = 64
X = Y = 224
data = ImageDataGenerator(vertical_flip = False, horizontal_flip = True, rotation_range = 5, width_shift_range = 0.1, height_shift_range = 0.1, samplewise_center=False, samplewise_std_normalization=False, rescale=1./255, data_format=None, validation_split=0.2)
training = data.flow_from_directory(directory = data_dir, target_size = (X, Y), color_mode="rgb",class_mode='categorical', batch_size=BATCH_SIZE, shuffle=True, seed=42, subset="training")
validation = data.flow_from_directory(directory = data_dir, target_size = (X, Y), color_mode="rgb",class_mode='categorical', batch_size=BATCH_SIZE, shuffle=False, seed=42, subset="validation")



Found 4098 images belonging to 4 classes.
Found 1023 images belonging to 4 classes.


Step 5: Defining The Squeeze and Excitation Block (SE Block)

In [ ]:
K.set_image_data_format('channels_last')

def squeeze_excite_block(input, ratio=16):
    ''' Create a squeeze-excite block
    Args:
        input: input tensor
        filters: number of output filters
        k: width factor
    Returns: a keras tensor
    '''
    init = input
    channel_axis = 1 if K.image_data_format() == "channels_first" else -1
    filters = init.shape[channel_axis]
    se_shape = (1, 1, filters)

    se = GlobalAveragePooling2D()(init)
    se = Reshape(se_shape)(se)
    se = Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)

    if K.image_data_format() == 'channels_first':
        se = Permute((3, 1, 2))(se)

    x = multiply([init, se])
    return x

Step 6: Select the feature Descriptor (NasNetMobile, VGG16, Resnet50) and train the model.

In [ ]:

# Select the feature Descriptor by commenting and uncommenting:

# VGG16:
# model= tf.keras.applications.VGG16(input_shape= (X, Y, 3), include_top= False, weights='imagenet', input_tensor=None, pooling='None', classes=1000)

# NasNetMobile:
# model= tf.keras.applications.NASNetMobile(input_shape= (X, Y, 3), include_top= False, weights='imagenet', input_tensor=None, pooling=None, classes=1000)

# Resnet50:
model= tf.keras.applications.ResNet50(input_shape= (X, Y, 3), include_top= False, weights='imagenet', input_tensor=None, pooling=None, classes=1000)


# Defining the MLP Classifier and building the model:
model.trainable = True
x = model.output
x=squeeze_excite_block(x, ratio=6)
x=GlobalAveragePooling2D()(x)
x = Flatten()(x)
x = Dense(512,activation='relu',kernel_initializer='he_uniform')(x)
x = keras.layers.Dropout(0.1)(x)
x = Dense(128,activation='relu',kernel_initializer='he_uniform')(x)
x = Dense(32,activation='relu',kernel_initializer='he_uniform')(x)
predictions = Dense(4,activation='softmax')(x)
model = keras.models.Model(inputs=model.input, outputs=predictions)


Step 7: Compiling the model

In [ ]:
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss


METRICS = [tf.keras.metrics.BinaryAccuracy(name='accuracy'),tf.keras.metrics.Precision(name='precision'),tf.keras.metrics.Recall(name='recall'),  tf.keras.metrics.AUC(name='auc')]
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-6),loss=weighted_categorical_crossentropy(np.array([5121/717, 5121/52, 5121/2560, 5121/1792]).astype(float)),metrics=METRICS)

Step 8 : Training the model

In [ ]:
callback_model= keras.callbacks.ModelCheckpoint( '/content' + '/model.h5', monitor= "val_auc", verbose= 1, save_best_only= True, mode= "max")
callback_CSV= keras.callbacks.CSVLogger('/content' + "/model.log", append=True)

# Training the model
history = model.fit_generator(training, steps_per_epoch= len(training),  epochs=500, verbose=1, validation_data=validation, validation_steps= len(validation), callbacks=[callback_model, callback_CSV])

Step 9: Plott the graph of loss and AUC of the network.

In [ ]:
def MovingAverage (arr,window_size):
  i = 0
  moving_averages = []
  while i < len(arr) - window_size + 1:
    
      # Calculate the average of current window
      window_average = round(np.sum(arr[
        i:i+window_size]) / window_size, 2)
        
      # Store the average of current
      # window in moving average list
      moving_averages.append(window_average)
        
      # Shift window to right by one position
      i += 1
    
  return moving_averages


window_size = 5

import pandas as pd
data = pd.read_csv('/content/model.log', delimiter = ",", header = None)
epoch = np.array(data[0][1:]).astype(float) + 1
auc = MovingAverage(np.append(np.array(data[2][1:window_size]),np.array(data[2][1:])).astype(float),window_size)
loss = MovingAverage(np.append(np.array(data[3][1:window_size]),np.array(data[3][1:])).astype(float),window_size)
val_auc = MovingAverage(np.append(np.array(data[7][1:window_size]),np.array(data[7][1:])).astype(float),window_size)
val_loss = MovingAverage(np.append(np.array(data[8][1:window_size]),np.array(data[8][1:])).astype(float),window_size)


print(len(epoch))
print(len(auc))

plt.figure(figsize=(10, 10))
plt.subplot(2, 1, 1)
plt.plot(epoch, auc, label='Training AUC', color='r')
plt.plot(epoch, val_auc, label='Validation AUC', color='b')

plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(loc='lower right', fontsize=13)
plt.ylabel('AUC', fontsize=16, weight='bold')
plt.title('Training & Validation AUC.', fontsize=16, weight='bold')


plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss', color='r')
plt.plot(val_loss, label='Validation Loss', color='b')
plt.xticks(fontsize=14)
plt.yticks(fontsize=14)
plt.legend(loc='upper right', fontsize=13)
plt.ylabel('Cross Entropy', fontsize=16, weight='bold')
plt.title('Training & Validation Loss', fontsize=15, weight='bold')
plt.xlabel('Epoch', fontsize=15, weight='bold')
plt.show()

Step 10: Calculating the Precision, Recall, and F1-score

In [ ]:
from sklearn.metrics import classification_report

def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss

def loss(y_true, y_pred):
    # scale predictions so that the class probas of each sample sum to 1
    y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
    # clip to prevent NaN's and Inf's
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    # calc
    loss = y_true * K.log(y_pred) * weights
    loss = -K.sum(loss, -1)
    return loss

# Loading the model
model= keras.models.load_model("/content/drive/MyDrive/model.h5", custom_objects={"weighted_categorical_crossentropy":weighted_categorical_crossentropy,"loss":loss})    

# Comput different metrics

Y_pred = model.predict(validation)
y_pred = np.argmax(Y_pred, axis=1)

print(classification_report(validation.classes, y_pred))

Step 11: Calculate and plot the Confusion Matrix

In [ ]:
def plot_confusion_matrix(cm, classes,normalize=False,title='Confusion matrix',cmap=plt.cm.Blues):
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title, weight='bold', fontsize=16)
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, fontsize=14)
    plt.yticks(tick_marks, classes, fontsize=14)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center", fontsize=12, weight='bold',
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label', fontsize=16, weight='bold')
    plt.xlabel('Predicted label', fontsize=16, weight='bold')

# Compute confusion matrix
cnf_matrix = confusion_matrix(validation.classes, y_pred)
np.set_printoptions(precision=2)
print(cnf_matrix)

# Plot normalized confusion matrix
plt.figure(figsize=(10, 10))
plot_confusion_matrix(cnf_matrix, classes=['MildDemented', 'ModerateDemented', 'NonDemented', 'VeryDemented'],normalize=True,title='Normalized Confusion Matrix')
plt.show()

Step 12: Plot the feature maps of some layers of the network for a sample input image

In [ ]:
# Plot the sample Image
plt.imshow(validation[0][0][0])

# Build a model for plotting the feature maps
layer_outputs = [layer.output for layer in model.layers]
feature_map_model = keras.models.Model(inputs=model.input, outputs=layer_outputs)
feature_maps  = feature_map_model.predict(np.expand_dims(validation[0][0][0],axis = 0))

# Select some layers of the network and plot the feature maps
feature_maps_list = [1,5,20]
square = 5
for fmap in feature_maps_list:
  print(fmap)
  ix = 1
  plt.figure(figsize=(10,10))
  for _ in range(square):
    for _ in range(square):
      ax = plt.subplot(square, square, ix)
      ax.set_xticks([])
      ax.set_yticks([])
      plt.imshow(feature_maps[fmap][0][:,:,ix-1])
      ix += 1
  plt.show()

Step 13: Copute the attention map for the actived class using GradCam

In [ ]:

from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm

def get_img_array(img_path, size):
    # `img` is a PIL image of size 299x299
    img = keras.preprocessing.image.load_img(img_path, target_size=size)
    # `array` is a float32 Numpy array of shape (299, 299, 3)
    array = keras.preprocessing.image.img_to_array(img)
    # We add a dimension to transform our array into a "batch"
    # of size (1, 299, 299, 3)
    array = np.expand_dims(array, axis=0)
    return array


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def save_and_display_gradcam(img, heatmap, cam_path="cam.jpg", alpha=0.6):
    # Load the original image
    # img = keras.preprocessing.image.load_img(img_path)
    # img = keras.preprocessing.image.img_to_array(img)

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]
    
    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = (jet_heatmap/255) * alpha + img
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    display(Image(cam_path))


# Generate class activation heatmap

layer_outputs = [layer.name for layer in model.layers]
last_conv_layer_name = layer_outputs[-13]
heatmap = make_gradcam_heatmap(np.expand_dims(validation[0][0][0],axis = 0), model, last_conv_layer_name)

# Display heatmap
plt.figure();plt.matshow(heatmap)

plt.figure();save_and_display_gradcam(validation[0][0][0], heatmap)


